## Timetabling Program for SMUS Middle School
#### Copyright © 2020 Hoshino Math Services

In [1]:
# import sys
# !{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# Import all the Data for Grades 6-8
InputFile = pd.ExcelFile("Middle School Input.xlsx") 
Grade6Matrix = pd.read_excel(InputFile, 'Grade6') 
Grade6Info = Grade6Matrix.values.tolist()
Grade7Matrix = pd.read_excel(InputFile, 'Grade7') 
Grade7Info = Grade7Matrix.values.tolist()
Grade8Matrix = pd.read_excel(InputFile, 'Grade8') 
Grade8Info = Grade8Matrix.values.tolist()


# Generate the set of Courses and Teachers at the Middle School
AllCourses = []
AllTeachers = []
for j in [1,5,9,13]:
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
            
            
# For each (Grade, Section, Course) triplet, determine the Teacher of that class.

GSCTeacher = [ [['Not Applicable' for c in range(40)] for s in range(5)] for g in range(10)]
for j in [1,5,9,13]:
    s = int((j+3)/4)
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[6][s][c] = Teacher
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[7][s][c] = Teacher
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[8][s][c] = Teacher

In [3]:
# Optimize assignment of courses to blocks


solver = pywraplp.Solver('SMUS Middle School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
start_time = time.time()

# Define our Grades, Sections, Days, Periods, Courses
Grades = [6,7,8]
Sections = [1,2,3,4]
Days = [1,2,3,4,5]
Periods = [1,2,3,4,5,6,7]
Courses = range(len(AllCourses))


# Define our five-dimensional Boolean Variables
x = {}
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                for c in Courses:
                    x[g,s,d,p,c] = solver.IntVar(0,1, 'x[%d,%d,%d,%d,%d]' % (g,s,d,p,c))

        
# CONSTRAINT 1: For each grade and each section, there is exactly one course each period.
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                solver.Add( sum(x[g,s,d,p,c] for c in Courses) == 1)
        
                        
# CONSTRAINT 2: Each course must be offered a set number of lessons in each week  
for s in Sections:
    for i in range(2, len(Grade6Info)-1):
        CourseName = Grade6Info[i][4*s-3]
        Lessons = Grade6Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[6,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade7Info)-1):
        CourseName = Grade7Info[i][4*s-3]
        Lessons = Grade7Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[7,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade8Info)-1):
        CourseName = Grade8Info[i][4*s-3]
        Lessons = Grade8Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[8,s,d,p,c] for d in Days for p in Periods) == Lessons)
        
        
# CONSTRAINT 3: Certain courses must be offered in certain blocks:
# Chapel on Tuesday Period 1, Assembly on Friday Period 1, XPLO on Friday Period 7
# Grade 7 Option in Period 5 (Monday, Thursday)
# Grade 8 Option in Period 5 (Tuesday, Friday)

for s in Sections:
    for g in Grades:        
        c = AllCourses.index('Chapel')
        solver.Add( x[g,s,2,1,c] == 1)
        c = AllCourses.index('Assembly')
        solver.Add( x[g,s,5,1,c] == 1)
        c = AllCourses.index('XPLO')
        solver.Add( x[g,s,5,7,c] == 1)

    c = AllCourses.index('Option')
    solver.Add( x[7,s,1,5,c] == 1)
    solver.Add( x[7,s,4,5,c] == 1)
    solver.Add( x[8,s,2,5,c] == 1)
    solver.Add( x[8,s,5,5,c] == 1)

    c = AllCourses.index('XPE')
    solver.Add( x[6,s,1,2,c] == 1)
    solver.Add( x[6,s,4,2,c] == 1)
    solver.Add( x[7,s,3,2,c] == 1)
    solver.Add( x[7,s,5,2,c] == 1)
    solver.Add( x[8,s,3,3,c] == 1)
    solver.Add( x[8,s,3,4,c] == 1)
    
    
# CONSTRAINT 4: No teacher can teach two different classes simultaneously
for Teacher in AllTeachers:
    Tset = []
    for g in Grades:
        for s in Sections:
            for c in Courses:    
                if GSCTeacher[g][s][c] == Teacher:
                    GradeSectionCourse = [g, s, c]
                    Tset.append(GradeSectionCourse)
    for d in Days:
        for p in Periods:
            solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]] 
                           for z in range(len(Tset))) <= 1)

                  
# CONSTRAINT 5: No teacher teaches more than three Academic lessons any morning or two Academic lessons
# any afternoon: Note that all courses are "academic" except for XPE and XPLO.
for Teacher in AllTeachers:
    Tset = []
    for g in Grades:
        for s in Sections:
            for c in Courses:    
                if GSCTeacher[g][s][c] == Teacher:
                    GradeSectionCourse = [g, s, c]
                    Tset.append(GradeSectionCourse)

    for d in Days:
        solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]]
                       for z in range(len(Tset)) for p in [1,2,3,4]) <= 3)
    for d in Days:
        solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]]
                       for z in range(len(Tset)) for p in [5,6,7]) <= 2)
       
      
    
# CONSTRAINT 6: A student can't have two lessons in the same class in non-consecutive periods.
# This constraint automatically implies that each class can be offered at most twice per day.
for p1 in Periods:
    for p2 in Periods:
        if p2-p1>1:
            for g in Grades:
                for s in Sections:
                    for d in Days:
                        for c in Courses:
                            solver.Add(x[g,s,d,p1,c] + x[g,s,d,p2,c] <=1)
                

# CONSTRAINT 7: Other than Hum, Math/Science, and Science, no multi-lesson class can be 
# offered more than once per day to any cohort.
for CourseName in ['French', 'Mandarin', 'Band', 'Strings', 'PE', 'Science', 'Art']:
    c = AllCourses.index(CourseName)
    for g in Grades:
        for s in Sections:
            for d in Days:
                solver.Add( sum(x[g,s,d,p,c] for p in Periods) <= 1)


# CONSTRAINT 8A: All Grade 6/7/8 students must have at least one Humanities class each day.
c = AllCourses.index('Humanities')
for g in Grades:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)

            
# CONSTRAINT 8B: All Grade 6/7 students must have at least one Math/Sci class each day.
c = AllCourses.index('Math/Science')
for g in [6,7]:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)

        
# CONSTRAINT 8C: All Grade 8 students must have Math four times a week
# either 5 singles or 1 double and 3 singles.  Thus, there cannot exist a pair of days
# in which Grade 8 math is not taught.
c = AllCourses.index('Math')
for d1 in Days:
    for d2 in Days:
        if d2-d1>0:
            for s in Sections:
                solver.Add(sum(x[8,s,d1,p,c]+x[8,s,d2,p,c] for p in Periods) >= 1)


# CONSTRAINT 9A: Band/Strings lessons must take place in the following periods:
# Monday 5-7, Tuesday 1-7, Wednesday 5-7, Thursday 1-7
for c in [AllCourses.index('Band'), AllCourses.index('Strings')]:
    for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,5,p,c] for p in [1,2,3,4,5,6,7]) == 0)
for c in [AllCourses.index('SharedBand'), AllCourses.index('SharedStrings')]:
    for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4]) == 0)
            solver.Add( sum(x[g,s,5,p,c] for p in [1,2,3,4,5,6,7]) == 0)
    
    
# CONSTRAINT 9B: Enns and Smith are not teaching SharedBand + SharedStrings
# at the same time as the lessons for Band and Strings

yy = AllCourses.index('SharedBand')
zz = AllCourses.index('SharedStrings')

for g in Grades:
    for s in Sections:
        for c in Courses:
            if GSCTeacher[g][s][c] == 'Enns':
                for d in Days:
                    for p in Periods:
                        solver.Add(x[g,s,d,p,c] + x[6,1,d,p,yy] <= 1)
                        solver.Add(x[g,s,d,p,c] + x[7,1,d,p,yy] <= 1)
                        solver.Add(x[g,s,d,p,c] + x[8,1,d,p,yy] <= 1)
            if GSCTeacher[g][s][c] == 'Smith':
                for d in Days:
                    for p in Periods:
                        solver.Add(x[g,s,d,p,c] + x[6,3,d,p,zz] <= 1)
                        solver.Add(x[g,s,d,p,c] + x[7,3,d,p,zz] <= 1)
                        solver.Add(x[g,s,d,p,c] + x[8,3,d,p,zz] <= 1)
for d in Days:
    for p in Periods:
        solver.Add(x[6,1,d,p,yy] + x[7,1,d,p,yy] <= 1)
        solver.Add(x[6,1,d,p,yy] + x[8,1,d,p,yy] <= 1)
        solver.Add(x[7,1,d,p,yy] + x[8,1,d,p,yy] <= 1)
        

# CONSTRAINT 9C: For each grade, Band/Strings take place in the same periods for cohorts A+C, and cohorts B+D
# And for each grade, SharedBand (cohorts A+B) and SharedStrings (cohorts C+D) take place at the exact same 
# period once per week, on a different day.

c1 = AllCourses.index('Band')
c2 = AllCourses.index('Strings')
c3 = AllCourses.index('SharedBand')
c4 = AllCourses.index('SharedStrings')

for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c1] == x[g,3,d,p,c2])
            solver.Add( x[g,2,d,p,c1] == x[g,4,d,p,c2])
            solver.Add( x[g,1,d,p,c3] == x[g,2,d,p,c3])
            solver.Add( x[g,2,d,p,c3] == x[g,3,d,p,c4])
            solver.Add( x[g,3,d,p,c4] == x[g,4,d,p,c4])
for g in Grades:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c1] + x[g,s,d,p,c3] for p in Periods) <= 1)
            solver.Add( sum(x[g,s,d,p,c2] + x[g,s,d,p,c4] for p in Periods) <= 1)
            
            
# CONSTRAINT 10: Choir lessons must take place in the following periods:
# Monday 1-2, Tuesday 1-7, Thursday 5-7, Friday 1-7.  Also no Grade 8 lessons during Period 7.
c = AllCourses.index('Choir')
for g in Grades:
        for s in Sections:
            solver.Add( sum(x[g,s,1,p,c] for p in [3,4,5,6,7]) == 0)
            solver.Add( sum(x[g,s,3,p,c] for p in [1,2,3,4,5,6,7]) == 0)
            solver.Add( sum(x[g,s,4,p,c] for p in [1,2,3,4]) == 0)
for s in Sections:
    for d in Days:
        solver.Add( x[8,s,d,7,c] == 0)
        
            
# CONSTRAINT 11: For each grade, French must take place in the same periods for cohorts A+C, and cohorts B+D
# And Band/Strings must take place in the same periods for cohorts A+C, and cohorts B+D
c = AllCourses.index('French')
for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c] == x[g,3,d,p,c])
            solver.Add( x[g,2,d,p,c] == x[g,4,d,p,c])
c1 = AllCourses.index('Band')
c2 = AllCourses.index('Strings')
for g in Grades:
    for d in Days:
        for p in Periods:
            solver.Add( x[g,1,d,p,c1] == x[g,3,d,p,c2])
            solver.Add( x[g,2,d,p,c1] == x[g,4,d,p,c2])

            
# CONSTRAINT 12A: Grade 8 students cannot have Art lessons on two consecutive days.
c = AllCourses.index('Art')
for s in Sections:
    for d in [1,2,3,4]:
        solver.Add(sum(x[8,s,d,p,c] + x[8,s,d+1,p,c] for p in Periods) <= 1) 

        
# CONSTRAINT 12B: Wilkins teaches at most 4 Art classes in a day.
c = AllCourses.index('Art')
for d in Days:
    solver.Add(sum(x[g,s,d,p,c] for g in Grades for s in Sections for p in Periods) <= 4)


        
# CONSTRAINT 13: Students cannot have French on three consecutive days.
c = AllCourses.index('French')
for g in Grades:
    for s in Sections:
        for d in [1,2,3]:
            solver.Add(sum(x[g,s,d,p,c] + x[g,s,d+1,p,c] + x[g,s,d+2,p,c] for p in Periods) <= 2)
            
            
# CONSTRAINT 14: Students cannot have Mandarin on two consecutive days.
c = AllCourses.index('Mandarin')
for g in Grades:
    for s in Sections:
        for d in [1,2,3,4]:
            solver.Add(sum(x[g,s,d,p,c] + x[g,s,d+1,p,c] for p in Periods) <= 1)
  
            
# CONSTRAINT 15A: Ensure students do not have PE on the same day as their XPE lesson.

c = AllCourses.index('PE')
for s in Sections:
    solver.Add( sum(x[6,s,1,p,c] for p in Periods) == 0)
    solver.Add( sum(x[6,s,4,p,c] for p in Periods) == 0)
    

# CONSTRAINT 15B: PE classes happen in pairs, where two cohorts are together.
# One of those paired PE classes must take place on Wednesday in Period 5.

c = AllCourses.index('PE')
for d in Days:
    for p in Periods:
        solver.Add( x[6,1,d,p,c] == x[6,3,d,p,c])
        solver.Add( x[6,2,d,p,c] == x[6,4,d,p,c])
solver.Add(x[6,1,3,5,c] + x[6,2,3,5,c] == 1)
    
               
# CONSTRAINT 15C: XPE teachers are not teaching XPE at the same time as their other classes
# Note we CANNOT have Danskin in Grade 6 XPE nor can we have Bell in Grade 7 XPE!

xpe = AllCourses.index('XPE')

for Teacher in ['Cunningham', 'Hollingworth', 'Kuklinski', 'Kim', 'Bell']:
    for g in Grades:
        for s in Sections:
            for c in Courses:
                if GSCTeacher[g][s][c] == Teacher:
                    for d in Days:
                        for p in Periods:
                            solver.Add(x[g,s,d,p,c] + x[6,1,d,p,xpe] <= 1)

for Teacher in ['Cunningham','Hollingworth', 'Kuklinski', 'Kim', 'Danskin']:
    for g in Grades:
        for s in Sections:
            for c in Courses:
                if GSCTeacher[g][s][c] == Teacher:
                    for d in Days:
                        for p in Periods:
                            solver.Add(x[g,s,d,p,c] + x[7,1,d,p,xpe] <= 1)

for Teacher in ['Cunningham',  'DeMerchant', 'Enns', 'Gains', 'Hollingworth', 'Kuklinski']:
    for g in Grades:
        for s in Sections:
            for c in Courses:
                if GSCTeacher[g][s][c] == Teacher:
                    for d in Days:
                        for p in Periods:
                            solver.Add(x[g,s,d,p,c] + x[8,1,d,p,xpe] <= 1)
        
    
        
# CONSTRAINT 16A: Computer Science (Option Course) cannot happen when any ADST is taking place.
c1 = AllCourses.index('ADST')
c2 = AllCourses.index('Option')
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                solver.Add(x[g,s,d,p,c1] + x[7,1,d,p,c2] <= 1)
                solver.Add(x[g,s,d,p,c1] + x[8,1,d,p,c2] <= 1)

    
# CONSTRAINT 16B: Two all-grade Option courses must take place in the morning, on the same days
# as the Option courses taking place in Period 5 that day.

c = AllCourses.index('Option')
for g in [7,8]:
    for d in Days:
        for p in Periods:
            solver.Add(x[g,1,d,p,c] == x[g,2,d,p,c2])
            solver.Add(x[g,2,d,p,c] == x[g,3,d,p,c2])
            solver.Add(x[g,3,d,p,c] == x[g,4,d,p,c2])
    solver.Add(sum(x[7,1,1,p,c] for p in [1,2,3,4]) == 1)
    solver.Add(sum(x[7,1,4,p,c] for p in [1,2,3,4]) == 1)
    solver.Add(sum(x[8,1,2,p,c] for p in [1,2,3,4]) == 1)
    solver.Add(sum(x[8,1,5,p,c] for p in [1,2,3,4]) == 1)


# CONSTRAINT 16C: Option teachers are not teaching this class at the same time 
# as their other classes

opt = AllCourses.index('Option')

for Teacher in ['Floyd', 'Zhang', 'Cunningham', 'Kuklinski']:
    for g in Grades:
        for s in Sections:
            for c in Courses:
                if GSCTeacher[g][s][c] == Teacher:
                    for d in Days:
                        for p in Periods:
                            solver.Add(x[g,s,d,p,c] + x[7,1,d,p,opt] <= 1)
                            solver.Add(x[g,s,d,p,c] + x[8,1,d,p,opt] <= 1)

                            
# CONSTRAINT 16D: Option teachers are not teaching in Period 6 and Period 7 during their Option days,
# since this would imply three afternoon classes in a row.

for Teacher in ['Floyd', 'Zhang', 'Cunningham', 'Kuklinski']:
    Tset = []
    for g in Grades:
        for s in Sections:
            for c in Courses:    
                if GSCTeacher[g][s][c] == Teacher:
                    GradeSectionCourse = [g, s, c]
                    Tset.append(GradeSectionCourse)
    for d in [1,2,4,5]:
        solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]]
                       for z in range(len(Tset)) for p in [6,7]) <= 1)
                               
                            
                            
# CONSTRAINT 17: Ensure Brambley off Tuesday afternoon
for g in Grades:
    for s in Sections:
        for c in Courses:
            if GSCTeacher[g][s][c] == 'Brambley':
                solver.Add(x[g,s,2,5,c]+x[g,s,2,6,c]+x[g,s,2,7,c] == 0)
        

# CONSTRAINT 18: Core courses (Humanities, French, Math, Science) must have certain 
# splits between AM and PM classes.
# 9 lessons must be 5-4 or 6-3 or 7-2 (Humanities, Math/Science)
# 8 lessons must be 5-3 or 6-2 (Humanities)
# 5 lessons must be 3-2 or 4-1 (Grade 8 Math)
# 3 lessons must be 2-1 or 1-2 (French, Science)

for s in Sections:
    for g in Grades:
        c = AllCourses.index('Humanities')
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [1,2,3,4]) >= 5)
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [5,6,7]) >= 2)
    for g in [6,7]:
        c = AllCourses.index('Math/Science')
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [1,2,3,4]) >= 5)
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [5,6,7]) >= 2)
    for g in [8]:
        c = AllCourses.index('Math')
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [1,2,3,4]) >= 3)
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [5,6,7]) >= 1)
    for g in Grades:
        c = AllCourses.index('French')
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [1,2,3,4]) >= 1)
        solver.Add(sum(x[g,s,d,p,c] for d in Days for p in [5,6,7]) >= 1)
                      
        
# CONSTRAINT 19: Catherine Cade off on Tuesday (fix one day of the week when she is off)
for g in Grades:
    for s in Sections:
        for c in Courses:
            if GSCTeacher[g][s][c] == 'Cade':
                solver.Add(sum(x[g,s,2,p,c] for p in Periods) == 0)
        
        
# RUN OPTIMIZATION

solver.Maximize(solver.Sum(x[g,s,d,p,c] for g in Grades for s in Sections for d in Days
                           for p in Periods for c in Courses))
sol = solver.Solve()

In [4]:
CohortNames = ["-", "A", "B", "C", "D"]
OurColumns = ["Period", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

c = AllCourses.index('SharedBand')
for g in Grades:
    for s in Sections:
        GSCTeacher[g][s][c] = 'Enns'
c = AllCourses.index('SharedStrings')
for g in Grades:
    for s in Sections:
        GSCTeacher[g][s][c] = 'Smith'

for g in Grades:
    for s in Sections:
        for c in [AllCourses.index('XPE'), AllCourses.index('Option'), AllCourses.index('XPLO')]:
            GSCTeacher[g][s][c] = 'Multiple Teachers'
        
        
Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[6,s,d,p,c].solution_value()==1:
                    CourseID = str(6) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[6][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade6 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[7,s,d,p,c].solution_value()==1:
                    CourseID = str(7) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[7][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade7 = pd.DataFrame(Timetable, columns=OurColumns)

Timetable = [ [ "-" for d in range(6)] for z in range(28)]
for z in range(28):
    Timetable[z][0] = "P" + str(1+int(z/4))
for d in Days:
    for p in Periods:
        for s in Sections:
            for c in Courses:
                if x[8,s,d,p,c].solution_value()==1:
                    CourseID = str(8) + CohortNames[s] + "-" + AllCourses[c]
                    TeacherID = " (" + GSCTeacher[8][s][c] + ")"
                    RowIndex = 4*(p-1)+(s-1)                        
                    Timetable[RowIndex][d] = CourseID + TeacherID
Grade8 = pd.DataFrame(Timetable, columns=OurColumns)

with pd.ExcelWriter('Optimal Middle School Timetable by Grade.xlsx') as writer:  
    Grade6.to_excel(writer, sheet_name='Grade6', index=False)
    Grade7.to_excel(writer, sheet_name='Grade7', index=False)
    Grade8.to_excel(writer, sheet_name='Grade8', index=False)

In [5]:
AllTeachers.sort()

CohortNames = ["-", "A", "B", "C", "D"]
OurColumns = ["Period"] + AllTeachers

for d in Days:
    Timetable = [ [ "-" for t in range(len(AllTeachers)+1)] for z in range(7)]
    for z in range(7):
        Timetable[z][0] = "P" + str(1+z)
    for g in Grades:
        for s in Sections:
            for p in Periods:
                for c in Courses:
                    if x[g,s,d,p,c].solution_value()==1:
                        CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                        TeacherID = GSCTeacher[g][s][c]
                        if TeacherID != 'Not Applicable' and TeacherID != 'Multiple Teachers':
                            t = AllTeachers.index(TeacherID)
                            Timetable[p-1][t+1] = CourseID
                        if TeacherID == 'Multiple Teachers':
                            if AllCourses[c] == 'Option':
                                for Teacher in ['Floyd', 'Zhang', 'Cunningham', 'Kuklinski']:
                                    CourseID = str(g) + 'E' + "-" + AllCourses[c]
                                    t = AllTeachers.index(Teacher)
                                    Timetable[p-1][t+1] = CourseID
                            if AllCourses[c] == 'XPE' and g==6:
                                for Teacher in ['Cunningham', 'Hollingworth', 'Kuklinski', 'Kim', 'Bell']:
                                    CourseID = str(g) + 'E' + "-" + AllCourses[c]
                                    t = AllTeachers.index(Teacher)
                                    Timetable[p-1][t+1] = CourseID
                            if AllCourses[c] == 'XPE' and g==7:
                                for Teacher in ['Cunningham', 'Hollingworth', 'Kuklinski', 'Kim', 'Danskin']:
                                    CourseID = str(g) + 'E' + "-" + AllCourses[c]
                                    t = AllTeachers.index(Teacher)
                                    Timetable[p-1][t+1] = CourseID
                            if AllCourses[c] == 'XPE' and g==8:
                                for Teacher in ['Cunningham', 'DeMerchant', 'Enns', 'Gains', 'Hollingworth', 'Kuklinski']:
                                    CourseID = str(g) + 'E' + "-" + AllCourses[c]
                                    t = AllTeachers.index(Teacher)
                                    Timetable[p-1][t+1] = CourseID                           
                                
    if d==1: Day1 = pd.DataFrame(Timetable, columns=OurColumns)
    if d==2: Day2 = pd.DataFrame(Timetable, columns=OurColumns)
    if d==3: Day3 = pd.DataFrame(Timetable, columns=OurColumns)
    if d==4: Day4 = pd.DataFrame(Timetable, columns=OurColumns)
    if d==5: Day5 = pd.DataFrame(Timetable, columns=OurColumns)


with pd.ExcelWriter('Optimal Middle School Timetable by Teacher.xlsx') as writer:  
    Day1.to_excel(writer, sheet_name='Day1', index=False)
    Day2.to_excel(writer, sheet_name='Day2', index=False)
    Day3.to_excel(writer, sheet_name='Day3', index=False)
    Day4.to_excel(writer, sheet_name='Day4', index=False)
    Day5.to_excel(writer, sheet_name='Day5', index=False)

In [6]:
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The complete timetabling algorithm ran in', round(solving_time,1), 'seconds')
    


Optimization Complete with Total Happiness Score of 420
The complete timetabling algorithm ran in 35.9 seconds
